In [16]:
import sys; sys.path.append('../../')
import numpy as np
import pandas as pd
from DataPreparation.DataPreparation import read_data
from sklearn.metrics import classification_report
from mlpath import mlquest as mlq
from sklearn.model_selection import cross_val_predict
from utils import load_hyperparameters, save_model, get_metrics
from ModelAnalysis import cross_validation
from sklearn.ensemble import RandomForestClassifier
model_name= 'RandomForest'

mlq.start_quest("RandomForest", table_dest="../../", log_defs=True)

# <font color="aqua">Random Forest</font> Model

### Read the data

In [17]:
x_data_d, y_data_d = mlq.l(read_data)(kind='Numerical', select=True)
x_data_d = x_data_d[['Weight', 'Height','Age']]
opt_params = load_hyperparameters(model_name)

### Initiate model

In [18]:
opt_params['n_estimators'] = 500
opt_params['max_features'] = 1
clf = mlq.l(RandomForestClassifier)(**opt_params)

### Evaluate Model Bias

In [19]:
clf.fit(x_data_d, y_data_d)
y_pred = clf.predict(x_data_d)
train_metrics = classification_report(y_data_d, y_pred, digits=3)
train_acc, train_wf1 = get_metrics(train_metrics)
print(train_metrics)

              precision    recall  f1-score   support

           0      0.984     0.984     0.984       190
           1      0.974     0.925     0.949       201
           2      0.969     0.993     0.981       406
           3      0.999     0.999     0.999       680

    accuracy                          0.985      1477
   macro avg      0.981     0.975     0.978      1477
weighted avg      0.985     0.985     0.985      1477



### Evaluate Model Generalization

In [21]:
kfold=cross_validation(clf, x_data_d, y_data_d, k=[10], n_repeats=[10], random_state=1,loo=False)

val_wf1 = kfold.get(f'10-Repeated 10-fold')[0]

10-Repeated 10-fold
0.9451


#### Save Model

In [22]:
save_model(model_name, clf)

### Tracking

In [23]:
mlq.log_metrics( train_wf1, val_wf1)
mlq.end_quest()
mlq.show_logs("RandomForest", table_dest="../../", last_k=6)